In [101]:
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [102]:
#########################
# HUMANS IMAGES
#########################

humans_path = '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images'

all_humans = []

for root, dirs, files in sorted(os.walk(humans_path)):

    for name in files:
        if not name.startswith("."):
            all_humans.append(str(root) + "/" + str(name))

all_humans = sorted(all_humans)

print("loaded %d files into human database" % len(all_humans))

loaded 36000 files into human database


In [103]:
#########################
# HUMANS ANNOTATIONS
#########################

humans_annotations_path = '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations'

female_humans_annotations = []
male_humans_annotations = []

for root, dirs, files in sorted(os.walk(humans_annotations_path)):
    for name in files:
        if not name.startswith("."):
            if "female" in root:
                female_humans_annotations.append(str(root) + "/" + str(name))
            else:
                male_humans_annotations.append(str(root) + "/" + str(name))

female_humans_annotations = sorted(female_humans_annotations)
male_humans_annotations = sorted(male_humans_annotations)

print("loaded %d files into female_humans_annotations database" % len(female_humans_annotations))
print(female_humans_annotations[0])
print(female_humans_annotations[1234])
print("loaded %d files into male_humans_annotations database" % len(male_humans_annotations))
print(male_humans_annotations[0])
print(male_humans_annotations[2999])

loaded 3000 files into female_humans_annotations database
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations/female/subject_mesh_0001_anno.json
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations/female/subject_mesh_1235_anno.json
loaded 3000 files into male_humans_annotations database
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations/male/subject_mesh_0001_anno.json
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations/male/subject_mesh_3000_anno.json


In [104]:
##############################################################
# DATA SORTING
##############################################################

##############################
# HUMANS
##############################

humans_plain_paths = []
humans_one_paths = []
humans_jeans_paths = []

for human in all_humans:
    if "ein" in human:
        humans_one_paths.append(human)
    elif "jeans" in human:
        humans_jeans_paths.append(human)
    else:
        humans_plain_paths.append(human)

##############################
# OUTPUT
##############################
print("Images loaded:")
print("----------------------------------------------------")
print("Examples:")
print("----------------------------------------------------")
print(humans_jeans_paths[0])
print(humans_plain_paths[0])
print(humans_one_paths[0])
print("----------------------------------------------------")
print("Numbers:")
print("----------------------------------------------------")
print(len(humans_jeans_paths))
print(len(humans_plain_paths))
print(len(humans_one_paths))

Images loaded:
----------------------------------------------------
Examples:
----------------------------------------------------
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images/200x200/pose0/female/subject_mesh_0001-m-jeans.png
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images/200x200/pose0/female/subject_mesh_0001-m.png
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images/200x200/pose0/female/subject_mesh_0001-m-ein.png
----------------------------------------------------
Numbers:
----------------------------------------------------
12000
12000
12000


In [111]:
from numpy.random import default_rng
import re

rng = default_rng()

# create 3 random arrays for 3 datasets to train/test the cnn
numbers_plain = rng.choice(12000, size=10000, replace=False)
numbers_jeans = rng.choice(12000, size=10000, replace=False)
numbers_one = rng.choice(12000, size=10000, replace=False)

##############################################################
# CONVERT IMAGES TO GRAYSCALE
##############################################################

# def get_female_ann(file):
#     id = get_id(file)-1
#     file = file[file.rfind('/')+1:]
#     # female_human_annotation = json.load(file)
#     with open(female_humans_annotations[id], 'r') as file:
#             # load from json
#             female_human_annotation = json.load(file)
#             sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
#             sub_df = sub_df.transpose()
#     return file

def get_male_ann(file):
    file = file[file.rfind('/')+1:]
    return file

def get_gender(filename):
    return filename[-5:-4]

def get_id(filename):
    return remove_non_int(filename[-17:])

def remove_non_int(str):
    return int(re.sub("[^0-9]", "", str))

# for file_path in humans_plain_paths:
#     print(file_path)
#     print(get_female_ann(file_path))
#     print(get_gender(file_path))
#     print(get_id(file_path))


# for female_humans_annotation_path in female_humans_annotations:
#     if not "DS_Store" in female_humans_annotation_path:
#         with open(female_humans_annotation_path, 'r') as file:
#             # load from json
#             female_human_annotation = json.load(file)
#             sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
#             sub_df = sub_df.transpose()
#
#             #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
#             humans_annotations_df = pd.concat([humans_annotations_df, sub_df])



# reduce number of images for testing purposes
n_images = 1000000

humans_plain_train, humans_plain_test = [], []
humans_plain_train_ann_df, humans_plain_test_ann_df = pd.DataFrame(), pd.DataFrame()

humans_jeans_grey_train, humans_jeans_grey_test = [], []
humans_jeans_grey_train_ann_df, humans_jeans_grey_test_ann_df = pd.DataFrame(), pd.DataFrame()

humans_one_grey_train, humans_one_grey_test = [], []
humans_one_grey_train_ann_df, humans_one_grey_test_ann_df = pd.DataFrame(), pd.DataFrame()

##############################
# HUMANS PLAIN
##############################

i = 0

for file_path in humans_plain_paths:
    human_plain = io.imread(file_path)
    id = get_id(file_path)-1
    gender = get_gender(file_path)

    if i in numbers_plain:
        humans_plain_train.append(human_plain)
        if gender == "f":
            with open(female_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                # print(human_annotation)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_plain_train_ann_df = pd.concat([humans_plain_train_ann_df, sub_df])
        else:
            with open(male_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                # print(human_annotation)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()
                # print(sub_df)

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_plain_train_ann_df = pd.concat([humans_plain_train_ann_df, sub_df])

    else:
        humans_plain_test.append(human_plain)
        if gender == "f":
            with open(female_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_plain_test_ann_df = pd.concat([humans_plain_test_ann_df, sub_df])
        else:
            with open(male_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_plain_test_ann_df = pd.concat([humans_plain_test_ann_df, sub_df])
    if i == n_images:
        break

    i += 1

print("1")
##############################
# HUMANS W TEXTURE FOREGROUND
##############################

i = 0

for file_path in humans_jeans_paths:
    human_texture_grey = color.rgb2gray(io.imread(file_path))
    id = get_id(file_path)-1
    gender = get_gender(file_path)

    if i in numbers_jeans:
        humans_jeans_grey_train.append(human_texture_grey)
        if gender == "f":
            with open(female_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_jeans_grey_train_ann_df = pd.concat([humans_jeans_grey_train_ann_df, sub_df])
        else:
            with open(male_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_jeans_grey_train_ann_df = pd.concat([humans_jeans_grey_train_ann_df, sub_df])
    else:
        humans_jeans_grey_test.append(human_texture_grey)
        if gender == "f":
            with open(female_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_jeans_grey_test_ann_df = pd.concat([humans_jeans_grey_test_ann_df, sub_df])
        else:
            with open(male_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_jeans_grey_test_ann_df = pd.concat([humans_jeans_grey_test_ann_df, sub_df])

    if i == n_images:
        break

    i += 1

print("2")
##############################
# HUMANS W SOLID FOREGROUND
##############################

i = 0

for file_path in humans_one_paths:
    human_rgb_grey = color.rgb2gray(io.imread(file_path))
    id = get_id(file_path)-1
    gender = get_gender(file_path)

    if i in numbers_one:
        humans_one_grey_train.append(human_rgb_grey)
        if gender == "f":
            with open(female_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_one_grey_train_ann_df = pd.concat([humans_one_grey_train_ann_df, sub_df])
        else:
            with open(male_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_one_grey_train_ann_df = pd.concat([humans_one_grey_train_ann_df, sub_df])
    else:
        humans_one_grey_test.append(human_rgb_grey)
        if gender == "f":
            with open(female_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_one_grey_test_ann_df = pd.concat([humans_one_grey_test_ann_df, sub_df])
        else:
            with open(male_humans_annotations[id], 'r') as file:
                # load from json
                human_annotation = json.load(file)
                sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
                sub_df = sub_df.transpose()

                #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
                humans_one_grey_test_ann_df = pd.concat([humans_one_grey_test_ann_df, sub_df])

    if i == n_images:
        break

    i += 1

print("3")

1
2
3


In [112]:
print(humans_one_grey_train_ann_df.columns)
print(humans_one_grey_train_ann_df.head())

Index(['chest_circumference', 'height', 'inseam', 'left_arm_length',
       'pelvis_circumference', 'right_arm_length', 'shoulder_width',
       'waist_circumference'],
      dtype='object')
   chest_circumference    height    inseam  left_arm_length  \
0             1.199463  1.934066  0.836919         0.651372   
0             1.022033  1.574711  0.593292         0.502548   
0             1.103838  1.907959  0.821567         0.619892   
0             1.361146  2.009109  0.850917         0.672863   
0             1.003971  1.542366  0.692716         0.505650   

   pelvis_circumference  right_arm_length  shoulder_width  waist_circumference  
0              1.167261          0.664282        0.456476             1.063651  
0              0.962600          0.521710        0.428650             0.856717  
0              1.069130          0.643452        0.442495             0.964508  
0              1.237328          0.678925        0.475370             1.240309  
0              0.944154  

In [113]:
print("humans_one dataset: " + str(len(humans_one_grey_train)) + ", " + str(len(humans_one_grey_test)))
print("humans_plain dataset: " + str(len(humans_plain_train)) + ", " + str(len(humans_plain_test)))
print("humans_jeans dataset: " + str(len(humans_jeans_grey_train)) + ", " + str(len(humans_jeans_grey_test)))



humans_one dataset: 10000, 2000
humans_plain dataset: 10000, 2000
humans_jeans dataset: 10000, 2000


In [114]:
##############################################################
# CREATE DATA FRAME OF IMAGE VECTORS AND ANNOTATIONS
##############################################################

##############################
# HUMAN PLAIN DATA FRAME
##############################

# tensor of images w.o. annotations
humans_plain_image_train_tensor = torch.tensor(humans_plain_train)
humans_plain_image_test_tensor = torch.tensor(humans_plain_test)
humans_plain_ann_train_tensor = torch.tensor(humans_plain_train_ann_df.values)
humans_plain_ann_test_tensor = torch.tensor(humans_plain_test_ann_df.values)

##############################
# HUMAN RGB DATA FRAME
##############################

# tensor of images w.o. annotations
humans_one_image_train_tensor = torch.tensor(humans_one_grey_train)
humans_one_image_test_tensor = torch.tensor(humans_one_grey_test)
humans_one_ann_train_tensor = torch.tensor(humans_one_grey_train_ann_df.values)
humans_one_ann_test_tensor = torch.tensor(humans_one_grey_test_ann_df.values)

##############################
# HUMAN TEXTURE DATA FRAME
##############################

# tensor of images w.o. annotations
humans_jeans_image_train_tensor = torch.tensor(humans_jeans_grey_train)
humans_jeans_image_test_tensor = torch.tensor(humans_jeans_grey_test)
humans_jeans_ann_train_tensor = torch.tensor(humans_jeans_grey_train_ann_df.values)
humans_jeans_ann_test_tensor = torch.tensor(humans_jeans_grey_test_ann_df.values)

############################################################
# STORE TENSORS IN FILE
############################################################

##############################
# PLAIN
##############################

torch.save(humans_plain_image_train_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_plain_image_train_tensor.pt')
torch.save(humans_plain_image_test_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_plain_image_test_tensor.pt')
torch.save(humans_plain_ann_train_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_plain_ann_train_tensor.pt')
torch.save(humans_plain_ann_test_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_plain_ann_test_tensor.pt')

##############################
# ONE COLOR
##############################

torch.save(humans_one_image_train_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_one_image_train_tensor.pt')
torch.save(humans_one_image_test_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_one_image_test_tensor.pt')
torch.save(humans_one_ann_train_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_one_ann_train_tensor.pt')
torch.save(humans_one_ann_test_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_one_ann_test_tensor.pt')

##############################
# JEANS
##############################

torch.save(humans_jeans_image_train_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_jeans_image_train_tensor.pt')
torch.save(humans_jeans_image_test_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_jeans_image_test_tensor.pt')
torch.save(humans_jeans_ann_train_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_jeans_ann_train_tensor.pt')
torch.save(humans_jeans_ann_test_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_jeans_ann_test_tensor.pt')


/var/folders/rl/xp7lnvk10sx90l166vcpd6p80000gn/T/ipykernel_26771/2711629265.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  humans_plain_image_train_tensor = torch.tensor(humans_plain_train)
